# Move and trim files recorded on Feb 18th, concert

Check and clear 

Plot and review measurements from equivital sensors as output by the Equivital Manager app.
(Every program produces different formated files and it's a pain.)

In [3]:
import sys
import os
import shutil
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import heartpy as hp

from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

## Definitions for data extraction

In [4]:
%load_ext autoreload
%autoreload 1
%aimport qex

In [3]:
%reload_ext autoreload

In [5]:
seperator = str('\\')
seperator

'\\'

# Scan for DATA files

All recordinds exported by Qiosk produce four CSV files and one SEM file. One of the CSVs is a DATA file that contains metadata and some essential signal quality checks on 15 s intervals.

We can ID instances of signal recordings from statistics taken out of the DATA file, so that gets extracted first

In [6]:
# location of the files when exported by QIOSK
path = "C:\\Users\\Public\\Documents\\Equivital\\Equivital Manager Wizard\\"#Extracted SEM Data\\" #"../Equivital/Equivital Manager Wizard/Extracted SEM Data/"
#os.listdir(path)

df_datafiles = qex.qiosk_recordings(path,'Present',seperator)
df_datafiles

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,VN102,3420426,230218,0,DATA-VN102-3420426-23021800.CSV,csv,70509,2023-02-18 13:25:57+00:00,2023-02-18 17:00:27+00:00,...,[VN102],3420426,VN102,NaN,0,100,0,1,Stationary,Upright
1,DATA,VN101,3420415,230218,0,DATA-VN101-3420415-23021800.CSV,csv,71160,2023-02-18 13:25:57+00:00,2023-02-18 17:00:27+00:00,...,[VN101],3420415,VN101,NaN,0,98,0,1,MovingSlowly,Upright
2,DATA,VN103,3420427,230218,0,DATA-VN103-3420427-23021800.CSV,csv,70869,2023-02-18 13:25:59+00:00,2023-02-18 17:00:29+00:00,...,[VN103],3420427,VN103,NaN,0,100,0,1,Stationary,Upright
3,DATA,VN104,3420456,230218,0,DATA-VN104-3420456-23021800.CSV,csv,70650,2023-02-18 13:26:01+00:00,2023-02-18 17:00:31+00:00,...,[VN104],3420456,VN104,NaN,0,100,0,1,Stationary,Prone
4,DATA,VN106,5022002,230218,0,DATA-VN106-5022002-23021800.CSV,csv,71093,2023-02-18 13:26:05+00:00,2023-02-18 17:00:35+00:00,...,[VN106],5022002,VN106,NaN,0,100,0,0,Stationary,Upright
5,DATA,VN105,3420755,230218,0,DATA-VN105-3420755-23021800.CSV,csv,70421,2023-02-18 13:26:05+00:00,2023-02-18 17:00:35+00:00,...,[VN105],3420755,VN105,NaN,0,98,0,1,Stationary,Upright
6,DATA,VN108,5022009,230218,0,DATA-VN108-5022009-23021800.CSV,csv,70636,2023-02-18 13:26:36+00:00,2023-02-18 17:00:51+00:00,...,[VN108],5022009,VN108,NaN,4,98,0,1,Stationary,Upright
7,DATA,VN107,5022006,230218,0,DATA-VN107-5022006-23021800.CSV,csv,70577,2023-02-18 13:26:36+00:00,2023-02-18 17:00:51+00:00,...,[VN107],5022006,VN107,NaN,0,97,0,1,Stationary,Upright
8,DATA,VN201,5022050,230218,0,DATA-VN201-5022050-23021800.CSV,csv,70558,2023-02-18 13:26:38+00:00,2023-02-18 17:00:53+00:00,...,[VN201],5022050,VN201,NaN,0,98,0,1,Stationary,Upright
9,DATA,VN202,5022051,230218,0,DATA-VN202-5022051-23021800.CSV,csv,70533,2023-02-18 13:26:41+00:00,2023-02-18 17:00:56+00:00,...,[VN202],5022051,VN202,NaN,0,99,0,1,Stationary,Upright


In [7]:
print('Number of DATA files: '+ str(len(df_datafiles)))
# key categories to structure of Qiosk output DATA files
print(df_datafiles['ID'].unique())
print(df_datafiles['DevName'].unique())
print(df_datafiles['Date'].unique())

Number of DATA files: 55
[3420426 3420415 3420427 3420456 5022002 3420755 5022009 5022006 5022050
 5022051 5022057 5022058 5022062 5022060 5022076 5022073 5022077 5022098
 5022100 5022101 5022110 5022104 5022117 5022126 5022127 5022131 5022138
 5022139 5022142 5022141 5023926 5022143 5022149 5022151 5022155 5022154
 5022156 5022164 5022163 5022161 5022166 5022173 5022184 5022179 5022198
 5022193 5022203 5022202 5022204 5022238 5022240 5022241 5022244 5022245
 5022256]
['VN102' 'VN101' 'VN103' 'VN104' 'VN106' 'VN105' 'VN108' 'VN107' 'VN201'
 'VN202' 'VN203' 'VN204' 'VN206' 'VN205' 'VN208' 'VN207' 'VN209' 'VN210'
 'VA301' 'VA302' 'VA304' 'VA303' 'VA305' 'VA306' 'VA307' 'VA308' 'LS401'
 'LS402' 'LS404' 'LS403' 'LS406' 'LS405' 'LS407' 'LS408' 'WW501' 'LS410'
 'WW502' 'WW505' 'WW504' 'WW503' 'WW506' 'WW507' 'BR602' 'BR601' 'BR604'
 'BR603' 'BR606' 'BR605' 'BR607' 'PC701' 'PC702' 'PC703' 'PC704' 'PC705'
 'PC707']
[230218]


# discard tiny

Use the Sort_Equivital_Qiosk_Export notebook to discard files 

## Define set of recordings to move

We sort files by day and time recording

In [12]:
# DATA files from recordings on a particular day, YYMMDD
recDate = 230218
s_files=df_datafiles.loc[df_datafiles['Date'] == recDate,:]


Additionally select according to start and stop recording times.

In [9]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-17 9:34:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']< t,:] 

In [10]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-17 07:00:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']> t,:] 

In [14]:
# recordings ended after a certain time
t = pd.to_datetime('2023-02-17 10:30:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecEnd']> t,:] 

In [14]:
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,VN102,3420426,230218,0,DATA-VN102-3420426-23021800.CSV,csv,70509,2023-02-18 13:25:57+00:00,2023-02-18 17:00:27+00:00,...,[VN102],3420426,VN102,NaN,0,100,0,1,Stationary,Upright
1,DATA,VN101,3420415,230218,0,DATA-VN101-3420415-23021800.CSV,csv,71160,2023-02-18 13:25:57+00:00,2023-02-18 17:00:27+00:00,...,[VN101],3420415,VN101,NaN,0,98,0,1,MovingSlowly,Upright
2,DATA,VN103,3420427,230218,0,DATA-VN103-3420427-23021800.CSV,csv,70869,2023-02-18 13:25:59+00:00,2023-02-18 17:00:29+00:00,...,[VN103],3420427,VN103,NaN,0,100,0,1,Stationary,Upright
3,DATA,VN104,3420456,230218,0,DATA-VN104-3420456-23021800.CSV,csv,70650,2023-02-18 13:26:01+00:00,2023-02-18 17:00:31+00:00,...,[VN104],3420456,VN104,NaN,0,100,0,1,Stationary,Prone
4,DATA,VN106,5022002,230218,0,DATA-VN106-5022002-23021800.CSV,csv,71093,2023-02-18 13:26:05+00:00,2023-02-18 17:00:35+00:00,...,[VN106],5022002,VN106,NaN,0,100,0,0,Stationary,Upright
5,DATA,VN105,3420755,230218,0,DATA-VN105-3420755-23021800.CSV,csv,70421,2023-02-18 13:26:05+00:00,2023-02-18 17:00:35+00:00,...,[VN105],3420755,VN105,NaN,0,98,0,1,Stationary,Upright
6,DATA,VN108,5022009,230218,0,DATA-VN108-5022009-23021800.CSV,csv,70636,2023-02-18 13:26:36+00:00,2023-02-18 17:00:51+00:00,...,[VN108],5022009,VN108,NaN,4,98,0,1,Stationary,Upright
7,DATA,VN107,5022006,230218,0,DATA-VN107-5022006-23021800.CSV,csv,70577,2023-02-18 13:26:36+00:00,2023-02-18 17:00:51+00:00,...,[VN107],5022006,VN107,NaN,0,97,0,1,Stationary,Upright
8,DATA,VN201,5022050,230218,0,DATA-VN201-5022050-23021800.CSV,csv,70558,2023-02-18 13:26:38+00:00,2023-02-18 17:00:53+00:00,...,[VN201],5022050,VN201,NaN,0,98,0,1,Stationary,Upright
9,DATA,VN202,5022051,230218,0,DATA-VN202-5022051-23021800.CSV,csv,70533,2023-02-18 13:26:41+00:00,2023-02-18 17:00:56+00:00,...,[VN202],5022051,VN202,NaN,0,99,0,1,Stationary,Upright


In [11]:
# show subset of files
s_files.iloc[0,:]

Signal                                                            DATA
DevName                                                          SPARE
ID                                                             5022299
Date                                                            230215
Session                                                              0
FileName                               DATA-SPARE-5022299-23021500.CSV
FileType                                                           csv
FileSize                                                         30091
RecStart                                     2023-02-15 06:39:12+00:00
FullLoc              C:\Users\Public\Documents\Equivital\Equivital ...
RecEnd                                       2023-02-15 08:08:57+00:00
Duration                                                        5385.0
BatteryStart                                                    4141.0
BatteryEnd                                                      4094.0
Batter

In [9]:
s_files.max()

Signal                                                            DATA
DevName                                                          WW507
ID                                                             5023926
Date                                                            230218
Session                                                              0
FileName                               DATA-WW507-5022173-23021800.CSV
FileType                                                           csv
FileSize                                                         72514
RecStart                                     2023-02-18 13:39:21+00:00
RecEnd                                       2023-02-18 17:18:51+00:00
Duration                                                       13170.0
BatteryStart                                                      4164
BatteryEnd                                                        4082
BatteryChange(mV)                                                  -70
FullLo

In [10]:
s_files.min()

Signal                                                            DATA
DevName                                                          BR601
ID                                                             3420415
Date                                                            230218
Session                                                              0
FileName                               DATA-BR601-5022179-23021800.CSV
FileType                                                           csv
FileSize                                                         69038
RecStart                                     2023-02-18 13:25:57+00:00
RecEnd                                       2023-02-18 17:00:27+00:00
Duration                                                       12660.0
BatteryStart                                                      3994
BatteryEnd                                                        3912
BatteryChange(mV)                                                 -111
FullLo

These recordings can be checked in View_Equivital_Qiosk_Export.

Now we move them

# Move selected files to suitable project folder

In [16]:
projectsFolder =  'D:\\' # where qiosk files are moved to after export
print(projectsFolder) # where the qiosk files are initially saved by the Equivital Qiosk program

D:\


In [17]:
# set project folder name
projectName = '230218_Lydo_Concerts'
projectPath = projectsFolder + projectName + '\\'

In [18]:
discardPath = 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\ToBeDeleted\\'

### Create folders for project to move files to
Check target location for data files to keep and to remove

In [19]:
# check if the folder exists for project and set up locations
if not os.path.isdir(projectsFolder + projectName):
    os.mkdir(projectsFolder + projectName)
    
if not os.path.isdir(projectsFolder + projectName + '\\' + 'SEM'):
    os.mkdir(projectsFolder + projectName + '\\' + 'SEM')
if not os.path.isdir(projectsFolder + projectName + '\\' + 'CSV'):
    os.mkdir(projectsFolder + projectName + '\\' + 'CSV')

# Set Interval to keep in CSV

We will cut the the CSV files to save on space and presever privacy. 
In this case, we are only taking the 

In [20]:
t1 = pd.to_datetime('2023-02-18 15:55:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]
t2 = pd.to_datetime('2023-02-18 17:13:00.00+0100')
#t2 = t1+pd.to_timedelta(240,'s')
[t1,t2]

[Timestamp('2023-02-18 15:55:00+0100', tz='pytz.FixedOffset(60)'),
 Timestamp('2023-02-18 17:13:00+0100', tz='pytz.FixedOffset(60)')]

In [21]:
(t2-t1).total_seconds()

4680.0

In [54]:
# test cutting

tic = time.time()

dataLoc = s_files.iloc[0,14]
matched = qex.matched_files(dataLoc,path,seperator) # outputs locations of csv 
for fi in matched:
    if not fi.lower().endswith('.sem'):
        fp = fi.split(seperator)
        fn = fp[-1].split('.')
        fn_new = fn[0]+'-trimmed.csv'
        print(fn_new)
        V = qex.cut_by_time(fi,t1,t2)
        print(V.columns)
        V.to_csv(projectPath + 'CSV\\' + fn_new)
        print(time.time()-tic)
    if eq_file_loc.lower().endswith('sem'):
        if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
            os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
        out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
        os.rename(eq_file_loc,out_f)
        print(time.time()-tic)

DATA-VN201-5022050-23021400-trimmed.csv
Index(['SENSOR ID', 'SUBJECT ID', 'SUBJECT AGE', 'HR(BPM)', 'HRC(%)',
       'BELT OFF', 'LEAD OFF', 'MOTION', 'BODY POSITION', 'BATTERY(mV)',
       'TEMPERATURE', 'BUTTON PRESS'],
      dtype='object')
0.4191780090332031
ECG-VN201-5022050-23021400-trimmed.csv
Index(['Lead 1', 'Lead 2'], dtype='object')
47.62759590148926
FASTACC-VN201-5022050-23021400-trimmed.csv
Index(['Vert Accelerometer', 'Lat Accelerometer', 'Long Accelerometer'], dtype='object')
90.35752868652344
RESPACC-VN201-5022050-23021400-trimmed.csv
Index(['Breathing'], dtype='object')
94.61357927322388
RR-VN201-5022050-23021400-trimmed.csv
Index(['Interbeat Interval (MS)'], dtype='object')
94.8211452960968


# Full Cut Save Move Discard

In [22]:
tac = time.time()

for i,row in s_files.iterrows():
    tic = time.time()
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,path,seperator)  # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                fn = fileName.split('.')
                fn_new = fn[0]+'-cut.csv'
                out_f = projectPath + 'CSV\\' + fn_new
                print(out_f)
                V = qex.cut_by_time(fi,t1,t2)
                V.to_csv(out_f) # write the trimmed csv to the external harddisk
                
                dout_f = discardPath + 'CSV\\' + fileName
                shutil.move(fi,dout_f) # move the exported csv files to the discard folder
                print(time.time()-tic)
        if fileName.lower().endswith('sem'):
            devName = row['DevName']
            if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
                os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
            out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
            shutil.move(fi,out_f)
            print(time.time()-tic)
            
    print([time.time()-tic, time.time()-tac])

dfiles = qex.qiosk_recordings(projectPath,projectName,seperator)
print(projectPath)
print(projectName)
dfiles

D:\230218_Lydo_Concerts\CSV\DATA-VN102-3420426-23021800-cut.csv
0.0573422908782959
D:\230218_Lydo_Concerts\CSV\ECG-VN102-3420426-23021800-cut.csv
16.738876342773438
D:\230218_Lydo_Concerts\CSV\FASTACC-VN102-3420426-23021800-cut.csv
34.550604820251465
D:\230218_Lydo_Concerts\CSV\RESPACC-VN102-3420426-23021800-cut.csv
36.98087215423584
D:\230218_Lydo_Concerts\CSV\RR-VN102-3420426-23021800-cut.csv
37.11971879005432
37.72781729698181
[37.72781729698181, 37.72881603240967]
D:\230218_Lydo_Concerts\CSV\DATA-VN101-3420415-23021800-cut.csv
0.051650047302246094
D:\230218_Lydo_Concerts\CSV\ECG-VN101-3420415-23021800-cut.csv
25.163008451461792
D:\230218_Lydo_Concerts\CSV\FASTACC-VN101-3420415-23021800-cut.csv
49.01647067070007
D:\230218_Lydo_Concerts\CSV\RESPACC-VN101-3420415-23021800-cut.csv
51.20690178871155
D:\230218_Lydo_Concerts\CSV\RR-VN101-3420415-23021800-cut.csv
51.34542226791382
51.947998046875
[51.947998046875, 89.67681407928467]
D:\230218_Lydo_Concerts\CSV\DATA-VN103-3420427-23021800-c

19.049778938293457
D:\230218_Lydo_Concerts\CSV\FASTACC-VN210-5022098-23021800-cut.csv
38.363842725753784
D:\230218_Lydo_Concerts\CSV\RESPACC-VN210-5022098-23021800-cut.csv
40.029587507247925
D:\230218_Lydo_Concerts\CSV\RR-VN210-5022098-23021800-cut.csv
40.16699719429016
40.771212339401245
[40.77221417427063, 788.0210294723511]
D:\230218_Lydo_Concerts\CSV\DATA-VA301-5022100-23021800-cut.csv
0.0461430549621582
D:\230218_Lydo_Concerts\CSV\ECG-VA301-5022100-23021800-cut.csv
19.673951387405396
D:\230218_Lydo_Concerts\CSV\FASTACC-VA301-5022100-23021800-cut.csv
38.78512763977051
D:\230218_Lydo_Concerts\CSV\RESPACC-VA301-5022100-23021800-cut.csv
40.53910040855408
D:\230218_Lydo_Concerts\CSV\RR-VA301-5022100-23021800-cut.csv
40.66344428062439
41.38504958152771
[41.38504958152771, 829.4060790538788]
D:\230218_Lydo_Concerts\CSV\DATA-VA302-5022101-23021800-cut.csv
0.03946828842163086
D:\230218_Lydo_Concerts\CSV\ECG-VA302-5022101-23021800-cut.csv
24.243757963180542
D:\230218_Lydo_Concerts\CSV\FASTA

42.91980338096619
D:\230218_Lydo_Concerts\CSV\RR-WW501-5022155-23021800-cut.csv
43.04413723945618
43.89870882034302
[43.89870882034302, 1520.8174669742584]
D:\230218_Lydo_Concerts\CSV\DATA-LS410-5022154-23021800-cut.csv
0.0370030403137207
D:\230218_Lydo_Concerts\CSV\ECG-LS410-5022154-23021800-cut.csv
19.028986930847168
D:\230218_Lydo_Concerts\CSV\FASTACC-LS410-5022154-23021800-cut.csv
36.54730701446533
D:\230218_Lydo_Concerts\CSV\RESPACC-LS410-5022154-23021800-cut.csv
38.73617768287659
D:\230218_Lydo_Concerts\CSV\RR-LS410-5022154-23021800-cut.csv
38.88305926322937
39.663593769073486
[39.663593769073486, 1560.4820613861084]
D:\230218_Lydo_Concerts\CSV\DATA-WW502-5022156-23021800-cut.csv
0.04506874084472656
D:\230218_Lydo_Concerts\CSV\ECG-WW502-5022156-23021800-cut.csv
22.34902834892273
D:\230218_Lydo_Concerts\CSV\FASTACC-WW502-5022156-23021800-cut.csv
39.681557178497314
D:\230218_Lydo_Concerts\CSV\RESPACC-WW502-5022156-23021800-cut.csv
41.40875220298767
D:\230218_Lydo_Concerts\CSV\RR-WW

20.966205596923828
D:\230218_Lydo_Concerts\CSV\FASTACC-PC704-5022244-23021800-cut.csv
39.47162747383118
D:\230218_Lydo_Concerts\CSV\RESPACC-PC704-5022244-23021800-cut.csv
41.43410897254944
D:\230218_Lydo_Concerts\CSV\RR-PC704-5022244-23021800-cut.csv
41.548102140426636
42.321218729019165
[42.321218729019165, 2312.9326198101044]
D:\230218_Lydo_Concerts\CSV\DATA-PC705-5022245-23021800-cut.csv
0.04294252395629883
D:\230218_Lydo_Concerts\CSV\ECG-PC705-5022245-23021800-cut.csv
19.640161514282227
D:\230218_Lydo_Concerts\CSV\FASTACC-PC705-5022245-23021800-cut.csv
41.70187449455261
D:\230218_Lydo_Concerts\CSV\RESPACC-PC705-5022245-23021800-cut.csv
43.45598912239075
D:\230218_Lydo_Concerts\CSV\RR-PC705-5022245-23021800-cut.csv
43.558165073394775
44.36323571205139
[44.36323571205139, 2357.2968425750732]
D:\230218_Lydo_Concerts\CSV\DATA-PC707-5022256-23021800-cut.csv
0.03199005126953125
D:\230218_Lydo_Concerts\CSV\ECG-PC707-5022256-23021800-cut.csv
21.994897603988647
D:\230218_Lydo_Concerts\CSV\F

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,VN104,3420456,230218,0,DATA-VN104-3420456-23021800-cut.csv,csv,26123,2023-02-18 14:55:01+00:00,2023-02-18 16:12:46+00:00,...,[VN104],3420456.0,VN104,NaN,81,98.0,0.0,0.0,MovingSlowly,Upright
1,DATA,WW502,5022156,230218,0,DATA-WW502-5022156-23021800-cut.csv,csv,25737,2023-02-18 14:55:01+00:00,2023-02-18 16:12:46+00:00,...,[WW502],5022156.0,WW502,NaN,74,98.0,0.0,0.0,Stationary,Upright
2,DATA,LS403,5022141,230218,0,DATA-LS403-5022141-23021800-cut.csv,csv,25842,2023-02-18 14:55:01+00:00,2023-02-18 16:12:46+00:00,...,[LS403],5022141.0,LS403,NaN,97,98.0,0.0,0.0,Stationary,Upright
3,DATA,VN204,5022058,230218,0,DATA-VN204-5022058-23021800-cut.csv,csv,25772,2023-02-18 14:55:01+00:00,2023-02-18 16:12:46+00:00,...,[VN204],5022058.0,VN204,NaN,75,98.0,0.0,0.0,Stationary,Upright
4,DATA,WW505,5022164,230218,0,DATA-WW505-5022164-23021800-cut.csv,csv,25846,2023-02-18 14:55:02+00:00,2023-02-18 16:12:47+00:00,...,[WW505],5022164.0,WW505,NaN,90,97.0,0.0,0.0,Stationary,Upright
5,DATA,VN209,5022077,230218,0,DATA-VN209-5022077-23021800-cut.csv,csv,25736,2023-02-18 14:55:02+00:00,2023-02-18 16:12:47+00:00,...,[VN209],5022077.0,VN209,NaN,84,98.0,0.0,0.0,Stationary,Upright
6,DATA,LS406,5023926,230218,0,DATA-LS406-5023926-23021800-cut.csv,csv,25818,2023-02-18 14:55:02+00:00,2023-02-18 16:12:47+00:00,...,[LS406],5023926.0,LS406,NaN,75,98.0,0.0,0.0,Stationary,Upright
7,DATA,PC705,5022245,230218,0,DATA-PC705-5022245-23021800-cut.csv,csv,24790,2023-02-18 14:55:02+00:00,2023-02-18 16:12:47+00:00,...,[PC705],5022245.0,PC705,NaN,36,75.0,0.0,0.0,Stationary,Side
8,DATA,BR601,5022179,230218,0,DATA-BR601-5022179-23021800-cut.csv,csv,25848,2023-02-18 14:55:03+00:00,2023-02-18 16:12:48+00:00,...,[BR601],5022179.0,BR601,NaN,73,97.0,0.0,0.0,Stationary,Upright
9,DATA,BR602,5022184,230218,0,DATA-BR602-5022184-23021800-cut.csv,csv,25771,2023-02-18 14:55:03+00:00,2023-02-18 16:12:48+00:00,...,[BR602],5022184.0,BR602,NaN,77,98.0,0.0,0.0,Stationary,Upright


In [3]:
1758/60

29.3

In [82]:
dfiles = qex.qiosk_recordings(projectPath,projectName,seperator)
print(projectPath)
print(projectName)
dfiles

D:\230214_Equivital\
230214_Equivital


,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,VA304,5022110,230214,0,DATA-VA304-5022110-23021400-cut.csv,csv,80491,2023-02-14 08:55:02+00:00,2023-02-14 12:59:47+00:00,...,[VA304],5022110.0,VA304,NaN,63,98.0,0.0,0.0,Stationary,Upright
1,DATA,BR604,5022198,230214,101,DATA-BR604-5022198-23021400_0-cut.csv,csv,80215,2023-02-14 08:55:04+00:00,2023-02-14 12:59:49+00:00,...,[BR604],5022198.0,BR604,NaN,65,98.0,0.0,0.0,Stationary,Upright
2,DATA,VN205,5022060,230214,0,DATA-VN205-5022060-23021400-cut.csv,csv,80246,2023-02-14 08:55:04+00:00,2023-02-14 12:59:49+00:00,...,[VN205],5022060.0,VN205,NaN,63,99.0,0.0,0.0,Stationary,Upright
3,DATA,PC702,5022240,230214,0,DATA-PC702-5022240-23021400-cut.csv,csv,80308,2023-02-14 08:55:04+00:00,2023-02-14 12:59:49+00:00,...,[PC702],5022240.0,PC702,NaN,70,98.0,0.0,0.0,Stationary,Upright
4,DATA,VN202,5022051,230214,0,DATA-VN202-5022051-23021400-cut.csv,csv,80336,2023-02-14 08:55:04+00:00,2023-02-14 12:59:51+00:00,...,[VN202],5022051.0,VN202,NaN,84,99.0,0.0,0.0,Stationary,Upright
5,DATA,PC703,5022241,230214,0,DATA-PC703-5022241-23021400-cut.csv,csv,80996,2023-02-14 08:55:05+00:00,2023-02-14 12:59:50+00:00,...,[PC703],5022241.0,PC703,NaN,74,100.0,0.0,0.0,Stationary,Upright
6,DATA,LS401,5022138,230214,0,DATA-LS401-5022138-23021400-cut.csv,csv,80302,2023-02-14 08:55:05+00:00,2023-02-14 12:59:50+00:00,...,[LS401],5022138.0,LS401,NaN,70,99.0,0.0,0.0,Stationary,Upright
7,DATA,PC701,5022238,230214,0,DATA-PC701-5022238-23021400-cut.csv,csv,80391,2023-02-14 08:55:05+00:00,2023-02-14 12:59:50+00:00,...,[PC701],5022238.0,PC701,NaN,80,98.0,0.0,0.0,Stationary,Upright
8,DATA,WW502,5022156,230214,0,DATA-WW502-5022156-23021400-cut.csv,csv,80378,2023-02-14 08:55:06+00:00,2023-02-14 12:59:51+00:00,...,[WW502],5022156.0,WW502,NaN,70,98.0,0.0,0.0,Stationary,Upright
9,DATA,LS403,5022141,230214,0,DATA-LS403-5022141-23021400-cut.csv,csv,80401,2023-02-14 08:55:07+00:00,2023-02-14 12:59:52+00:00,...,[LS403],5022141.0,LS403,NaN,88,98.0,0.0,0.0,Stationary,Upright


## Troubleshooting

In [69]:
shutil.move(fi,out_f)

'D:\\230214_Equivital\\SEM\\VN101\\23021400.SEM'

In [11]:
dfiles = qex.qiosk_recordings(discardPath, 'ToBeDiscarded',seperator)
print(len(dfiles))

585
